In [1]:
from pandas import DataFrame
import nltk, re, pprint
from collections import defaultdict

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
    
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn import grid_search
import random
random.seed(2017)

C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\Ambar Gupta\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# http://scikit-learn.org/stable/modules/feature_extraction.html
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

# https://github.com/scikit-learn/scikit-learn/issues/1156
# Snowball stemmers could be used as a dependency
from nltk.stem import SnowballStemmer

class build_stemmer(object):
    def __init__(self):
        self.wns = SnowballStemmer('english')
    def __call__(self, doc):
        return [self.wns.stem(t) for t in word_tokenize(doc)]    
    
# http://www.nltk.org/howto/stem.html 
# https://github.com/nltk/nltk/issues/1581
# pip install nltk==3.2.1
from nltk.stem.porter import *

class build_stemmer2(object):
    def __init__(self):
        self.wns = PorterStemmer()
    def __call__(self, doc):
        return [self.wns.stem(t) for t in word_tokenize(doc)]        
    
class lemma_stemmer(object):
    def __init__(self):
        self.wns = SnowballStemmer('english')
        self.wnl = WordNetLemmatizer() 
    def __call__(self, doc):
        return [self.wnl.lemmatize(self.wns.stem(t)) for t in word_tokenize(doc)]    

In [4]:
df = DataFrame.from_csv('train.tsv',sep='\t')
df.head()

,description
tags,
licence-needed supervising-job 5-plus-years-experience-needed,THE COMPANY Employer is a midstream service...
2-4-years-experience-needed salary full-time-job,ICR Staffing is now accepting resumes for Indu...
part-time-job,This is a great position for the right person....
licence-needed,A large multi-specialty health center is expan...
5-plus-years-experience-needed full-time-job bs-degree-needed supervising-job,JOB PURPOSE: The Account Director is respon...


In [5]:
df2 = DataFrame.from_csv('test.tsv',sep='\t')
df2.head(n=2)

""
description
"Integrity Home Care has an opening on our Leadership Team for a Salaried Private Care Nursing Supervisor. This position requires a MO & KS Licensed RN. Candidate must be able to work in a flexible environment with direct patient care experience as well as community health education, and one on one teaching experience. This position will be working with pediatric and senior patience in a home environment and hours will be mainly from 8am to 5pm, but candidate will need to be flexible as needed. If you enjoy the one on one interaction with patients, and working to fulfill their long term needs, start with Integrity today! We are a dynamic and progressive Christian based organization that offers competitive pay and excellent benefits including vacation and sick time, Health, dental, short term disability and life insurance."
"We're looking for a precision grinder with at least 10 years of experience. Prior experience grinding on carburzied parts is also needed. *Job Description for Precision Grinding Machinist:* * Set up and operate a variety of manual and CNC grinders for production, R&D and tool repair work. Grinding work will include, but not be limited to, steps, angles, contours and threads. Selects proper speeds, feeds and wheels. This position will be working in an aerospace manufacturing facility. *Job Requirements for Precision Grinding Machinist:* Must have at least 10 years experience with the following types of grinders: * Centerless * Internal * External * Surface * Thread * Universal Base Pay $35.00 - $40.00 / Hour Other Pay - O.T. & PER DIEM These positions are temporary and require travel. We pay your travel expenses. In return, you will have the opportunity to work up to 60 hours per week. You will receive weekly per diem of $595 ($85/day) for food and lodging. This is not an opportunity to permanently relocate because these positions are temporary. Other Requirements for Precision Grinding Machinist: * Felony/drug free and submit to a background check * Physical ability to work overtime – up to 60 hour work week * High Security Facility – Must be a US citizen Strom Engineering provides temporary and contract employment opportunities for general laborers, engineers, machinists, mechanics, technicians & welders."


In [6]:
tags = ["part-time-job", "full-time-job", "hourly-wage", "salary", "associate-needed", "bs-degree-needed", "ms-or-phd-needed",
        "licence-needed","1-year-experience-needed", "2-4-years-experience-needed", "5-plus-years-experience-needed",
        "supervising-job"]

In [7]:
def preproc(tag,df,df2):
    x_train = []
    y_train = []
    for index,row in df.iterrows():
        x_train.append(row.description.lower())
        if type(row.name) == str and  tag in row.name:
            y_train.append(1)
        else:
            y_train.append(0)
            
    docs_new = []
    for index, row in df2.iterrows():
        docs_new.append(row.name.lower())
    return x_train, y_train, docs_new

In [8]:
def pred(tag, x_train, y_train, docs_new):
    # http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
    #count_vect = CountVectorizer()
    count_vect = CountVectorizer(tokenizer=build_stemmer(), ngram_range=(1, 3))
    X_train_counts = count_vect.fit_transform(x_train)
    X_train_counts.shape
    
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
    X_train_tf = tf_transformer.transform(X_train_counts)
    X_train_tf.shape

    tfidf_transformer = TfidfTransformer()
    X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
    X_train_tfidf.shape


    clf = xgb.XGBClassifier()
    '''
    param_grid = {
        'n_estimators': [500],
        'learning_rate': [0.05],
        'max_depth': [5, 7],
        'subsample': [0.7, 0.8],
        'colsample_bytree': [0.75, 0.85],
    }
    '''

    param_grid = {
        'scale_pos_weight' : [2, 2.5, 3, 3.5],
        'n_estimators': [100, 150, 200],
    }

    
    model = grid_search.GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=5, cv=3, verbose=20, scoring = 'f1_micro')
    model.fit(X_train_tfidf, y_train)       
    X_new_counts = count_vect.transform(docs_new)
    X_new_tfidf = tfidf_transformer.transform(X_new_counts)
    

    predicted = model.predict(X_new_tfidf)


    print(sum(predicted))
    return predicted

In [9]:
output = [""]*2921
for i,tag in enumerate(tags):
    x_train,y_train,docs_new = preproc(tag,df,df2)
    output = pred(tag,x_train,y_train,docs_new)
    for j, item in enumerate(output):
        if item == 1:
            if output[j] == "":
                output[j] = tag
            else:
                output[j] += " " + tag
            
for i, item in enumerate(outputs):
    if item == "":
        output[i] = " "

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=5)]: Done   1 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done   2 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done   3 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done   4 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done   5 tasks      | elapsed:  5.6min
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed: 10.5min
[Parallel(n_jobs=5)]: Done   7 tasks      | elapsed: 10.5min
[Parallel(n_jobs=5)]: Done   8 tasks      | elapsed: 10.6min
[Parallel(n_jobs=5)]: Done   9 tasks      | elapsed: 10.7min
[Parallel(n_jobs=5)]: Done  10 tasks      | elapsed: 10.7min
[Parallel(n_jobs=5)]: Done  11 tasks      | elapsed: 15.6min
[Parallel(n_jobs=5)]: Done  12 tasks      | elapsed: 15.6min
[Parallel(n_jobs=5)]: Done  13 tasks      | elapsed: 18.0min
[Parallel(n_jobs=5)]: Done  14 tasks      | elapsed: 18.2min
[Parallel(n_jobs=5)]: Done  15 tasks      | elapsed: 18.2min
[Parallel(n_jobs=5)]: Done  16 tasks      | elapsed: 23.0min
[Parallel(n_jobs=5)]: Do

180


NameError: name 'outputs' is not defined